In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
fashion_mnist=keras.datasets.fashion_mnist

In [3]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
train_images=train_images/255.0
test_images=test_images/255.0

In [5]:
train_images[0].shape

(28, 28)

In [6]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model (hp):
     model =keras.Sequential([
        keras.layers.Conv2D(
                 filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                 kernel_size=hp.Choice( 'conv_1_kernel', values = [3,5]),
                 activation='relu',
                input_shape=(28,28,1)
         ),
        keras.layers.Conv2D(
                   filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
                   kernel_size=hp.Choice('conv_2_kernel', values= [3,5]),
                  activation='relu'
        ),
        keras.layers.Flatten(),
        keras.layers.Dense(
                  units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
                  activation='relu'
        ),
        keras.layers.Dense(10, activation='softmax')#output layer
       ])
     model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])), 
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
     return model

In [13]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 5.5MB/s 


In [14]:
import kerastuner as kt

In [16]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [17]:
 tuner_search=RandomSearch(build_model,
                               objective='val_accuracy',
                               max_trials=5,directory='output',project_name="Mnist Fashion")

In [18]:
 tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 04m 22s]
val_accuracy: 0.8761666417121887

Best val_accuracy So Far: 0.9126666784286499
Total elapsed time: 00h 52m 52s
INFO:tensorflow:Oracle triggered exit


In [21]:
 model=tuner_search.get_best_models(num_models=1)[0]

In [22]:
 model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 128)       1280      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        36896     
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1179712   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,218,538
Trainable params: 1,218,538
Non-trainable params: 0
_________________________________________________________________
